In [1]:
! pip install openai
! pip install wheel
! pip install hazm
! pip install transformers
! pip install sentencepiece
! pip install -U numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.5/367.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.1 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.25.1 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.6 MB/s eta 0:00:00


In [1]:
!gdown 1ICX68P6OUWtWicaYH2Gao2flcFX8_Xh3
!gdown 1qjnoGotE3oN-BjF9S90fIpakWbq25QGc

Downloading...
From: https://drive.google.com/uc?id=1ICX68P6OUWtWicaYH2Gao2flcFX8_Xh3
To: /content/eval.json
100% 1.78M/1.78M [00:00<00:00, 132MB/s]
Downloading...
From: https://drive.google.com/uc?id=1qjnoGotE3oN-BjF9S90fIpakWbq25QGc
To: /content/persian-zero.csv
100% 280k/280k [00:00<00:00, 94.1MB/s]


In [2]:
import time
import random
from hazm import stopwords_list
import numpy as np
import pandas as pd
from tqdm import tqdm
from hazm import *

In [ ]:
import openai
openai.api_key  = "..."

def get_completion(prompt, model="gpt-3.5-turbo"): # Andrew mentioned that the prompt/ completion paradigm is preferable for this class
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

# Reading Comprehension

In [9]:
import json

f = open('eval.json',encoding='utf8')
data = json.load(f)

with open("eval-data.json", "w",encoding='utf8') as write_file:
  json_string = json.dumps(data, ensure_ascii=False,indent=1).encode('utf8')
  write_file.write(json_string.decode())

rc_data = []
for item in data['data']:
  for paragraph in item['paragraphs']:
    context = paragraph['context']
    for qa_item in paragraph['qas']:
      question = qa_item['question']
      answers = []
      for answer in qa_item['answers']:
        answers.append(answer['text'])
      rc_data.append({"context":context,"question":question,"answers":answers})

In [10]:
random.seed(42)
samples = random.sample(rc_data, k=200)

In [11]:
samples[:2]

[{'context': 'هیدروسفالی(به انگلیسی: Hydrocephalus) در پزشکی به وضعیتی گفته می\u200cشود که مایع مغزی-نخاعی تولید شده در شبکه کوروئید٬ بیش از اندازه در بطن\u200cهای مغزی و دیگر حفره\u200cهای مغز انباشته گردد و منجر به افزایش حجم آن\u200cها شود. این افزایش حجم اشغال شده توسط مایع در سیستم عصبی مرکزی و بویژه مغز می\u200cتواند ناشی از اختلال در توزیع٬ تشکیل٬ جریان یا جذب مایع مغزی-نخاعی باشد و پی\u200cآمد آن افزایش فشار درون جمجمه و آسیب\u200cهای مغزی و حتی مرگ است. هیدروسفالی در کودکان زیر دو سال به\u200cخاطر بازبودن سوچورهای کرانیکال(درزهای استخوان جمجمه) منجربه بزرگ شدن غیرطبیعی جمجمه شده ولی در بزرگ\u200cسالان با اختلال در راه رفتن و حافظه نمود میابد. هیدروسفالی در کودکان بیشتر بعلت ناهنجاری مادرزادی کیاری*٬ تنگی در مجرای مغزی(سیلویوس) مادرزادی یا خونریزی و عفونت نوزادی داخل بطنی پدید می\u200cآید. بزرگ شدن بالای سر همراه\u200cبا نازک شدن استخوان\u200cهای جمجمه و جدا شدن سوچورها و در موارد پیشرفته\u200cتر آتروفی اپتیک و اختلال در حرکت چشم\u200cها و مشکل در تعادل و راه رفتن کودک٬ دیده می

In [12]:
print(len(samples))

200


In [13]:
sample_q = [sample['question'] for sample in samples]
sample_ans = [sample['answers'] for sample in samples]

In [14]:
others = [rc_data[i] for i in range(len(rc_data)) if ((rc_data[i]['question'] not in sample_q) and (rc_data[i]['answers'] not in sample_ans))]

In [15]:
random.seed(42)
three_shot = random.sample(others, k=3)

In [16]:
three_shot

[{'context': 'شما می توانید از ویتامین E برای رفع چین و چروک پیشانی و دیگر نشانه های پیری استفاده کنید. این ویتامین قدرت این را دارد که آثار ناشی از نور خورشید را برگرداند. حتی تولید کلاژن را  نیز می تواند افزایش دهد که باعث افزایش کشش پوست شما می شود که مانع از نشانه هایی همچون چین و چروک صورت می گردد.',
  'question': 'ویتامین ای را چه موقع استفاده کنیم؟',
  'answers': ['برای رفع چین و چروک پیشانی و دیگر نشانه های پیری']},
 {'context': 'کلنگ ساخت این برج در دی ماه ۱۳۷۶ پس از سه سال مطالعات شناخت و امکان\u200cسنجی با نام یادمان (مرکز ارتباطات بین\u200cالمللی تهران) بر زمین\u200cخورد و در سال ۱۳۸۰ به پیشنهاد شورای اسلامی شهر تهران به مناسبت یکصدمین زادروز روح\u200cالله خمینی بنیان\u200cگذار جمهوری اسلامی ایران با نام برج میلاد تغییر یافت. ساخت این برج ۱۱ سال به درازا انجامید. در ۸ سال نخست تنها ۴۰ درصد از برج تکمیل شده بود، اما با سرعت بخشیدن به پروژه توسط محمدباقر قالیباف، شهردار تهران، ۶۰ درصد بعدی در ۳۰ ماه ساخته شد.',
  'question': 'چرا نام برج میلاد میلاد است؟',
  'answers': ['به م

In [ ]:
template = f"""
  سوال:
  q

  جواب:
  ans

"""
sample_string = ''.join(template.replace('q',s['question']).replace('ans',s['answers'][0]) for s in three_shot)

In [ ]:
print(f'''{sample_string}''')


  سوال:
  ویتامین ای را چه موقع استفاده کنیم؟

  جواب:
  برای رفع چین و چروک پیشانی و دیگر نشانه های پیری


  سوال:
  چرا نام برج میلاد میلاد است؟

  جواب:
  به مناسبت یکصدمین زادروز روح‌الله خمینی بنیان‌گذار جمهوری اسلامی ایران


  سوال:
  متفقین به کدام کشور لقب پل پیروزی دادند؟

  جواب:
  ایران




# get 200 prompt results

english prompt

In [ ]:
results = {"context":[],"question":[],"correct_ans":[],"model_ans":[]}
for i in range(0,200):
  context = samples[i]['context']
  question = samples[i]['question']
  correct_ans =  samples[i]['answers']
  print(f'question {i+1}')

  prompt = f"""
  In this task, you will be shown a Persian passage and question. You need to write an answer for the question. Try to keep your answers as short as possible.

  context:
  '''{context}'''

  question:
  '''{question}'''

  answer:
  """

  response = get_completion(prompt)
  print(prompt)
  print("model ans: ",response)
  print("correct ans: ",correct_ans)
  print("-------------------------------------------------------------------------------------")
  results['context'].append(context)
  results['question'].append(question)
  results['correct_ans'].append(correct_ans)
  results['model_ans'].append(response)

  time.sleep(22)

question 1

  In this task, you will be shown a Persian passage and question. You need to write a answer for the question. Try to keep your answers as short as possible.

  context:
  '''هیدروسفالی(به انگلیسی: Hydrocephalus) در پزشکی به وضعیتی گفته می‌شود که مایع مغزی-نخاعی تولید شده در شبکه کوروئید٬ بیش از اندازه در بطن‌های مغزی و دیگر حفره‌های مغز انباشته گردد و منجر به افزایش حجم آن‌ها شود. این افزایش حجم اشغال شده توسط مایع در سیستم عصبی مرکزی و بویژه مغز می‌تواند ناشی از اختلال در توزیع٬ تشکیل٬ جریان یا جذب مایع مغزی-نخاعی باشد و پی‌آمد آن افزایش فشار درون جمجمه و آسیب‌های مغزی و حتی مرگ است. هیدروسفالی در کودکان زیر دو سال به‌خاطر بازبودن سوچورهای کرانیکال(درزهای استخوان جمجمه) منجربه بزرگ شدن غیرطبیعی جمجمه شده ولی در بزرگ‌سالان با اختلال در راه رفتن و حافظه نمود میابد. هیدروسفالی در کودکان بیشتر بعلت ناهنجاری مادرزادی کیاری*٬ تنگی در مجرای مغزی(سیلویوس) مادرزادی یا خونریزی و عفونت نوزادی داخل بطنی پدید می‌آید. بزرگ شدن بالای سر همراه‌با نازک شدن استخوان‌های جمجمه و جدا شدن سوچو

In [ ]:
df = pd.DataFrame(results)
df.to_csv('english-prompt-zero.csv')

english prompt 1-shot

In [ ]:
template = f"""
  question:
  Q

  answer:
  ANS

"""
sample_string = ''.join(template.replace('Q',s['question']).replace('ANS',s['answers'][0]) for s in three_shot[0:1])
print(sample_string)


  question:
  ویتامین ای را چه موقع استفاده کنیم؟

  answer:
  برای رفع چین و چروک پیشانی و دیگر نشانه های پیری




In [ ]:
results = {"context":[],"question":[],"correct_ans":[],"model_ans":[]}
for i in range(0,200):
  context = samples[i]['context']
  question = samples[i]['question']
  correct_ans =  samples[i]['answers']
  print(f'question {i+1}')

  prompt = f"""
  In this task, you will be shown a Persian passage and question. You need to write a answer for the question. Try to keep your answers as short as possible.


  samples:
  {sample_string}
  context:
  '''{context}'''

  question:
  '''{question}'''

  answer:
  """

  response = get_completion(prompt)
  print(prompt)
  print("model ans: ",response)
  print("correct ans: ",correct_ans)
  print("-------------------------------------------------------------------------------------")
  results['context'].append(context)
  results['question'].append(question)
  results['correct_ans'].append(correct_ans)
  results['model_ans'].append(response)

  time.sleep(22)

question 1

  In this task, you will be shown a Persian passage and question. You need to write a answer for the question. Try to keep your answers as short as possible.


  samples:
  
  question:
  ویتامین ای را چه موقع استفاده کنیم؟

  answer:
  برای رفع چین و چروک پیشانی و دیگر نشانه های پیری


  context:
  '''هیدروسفالی(به انگلیسی: Hydrocephalus) در پزشکی به وضعیتی گفته می‌شود که مایع مغزی-نخاعی تولید شده در شبکه کوروئید٬ بیش از اندازه در بطن‌های مغزی و دیگر حفره‌های مغز انباشته گردد و منجر به افزایش حجم آن‌ها شود. این افزایش حجم اشغال شده توسط مایع در سیستم عصبی مرکزی و بویژه مغز می‌تواند ناشی از اختلال در توزیع٬ تشکیل٬ جریان یا جذب مایع مغزی-نخاعی باشد و پی‌آمد آن افزایش فشار درون جمجمه و آسیب‌های مغزی و حتی مرگ است. هیدروسفالی در کودکان زیر دو سال به‌خاطر بازبودن سوچورهای کرانیکال(درزهای استخوان جمجمه) منجربه بزرگ شدن غیرطبیعی جمجمه شده ولی در بزرگ‌سالان با اختلال در راه رفتن و حافظه نمود میابد. هیدروسفالی در کودکان بیشتر بعلت ناهنجاری مادرزادی کیاری*٬ تنگی در مجرای مغزی(سیلویو

In [ ]:
df = pd.DataFrame(results)
df.to_csv('rc-english-prompt-1shot.csv')

english promt 3-shot

In [ ]:
template = f"""
  question:
  Q

  answer:
  ANS

"""
sample_string = ''.join(template.replace('Q',s['question']).replace('ANS',s['answers'][0]) for s in three_shot)

In [ ]:
print(sample_string)


  question:
  ویتامین ای را چه موقع استفاده کنیم؟

  answer:
  برای رفع چین و چروک پیشانی و دیگر نشانه های پیری


  question:
  چرا نام برج میلاد میلاد است؟

  answer:
  به مناسبت یکصدمین زادروز روح‌الله خمینی بنیان‌گذار جمهوری اسلامی ایران


  question:
  متفقین به کدام کشور لقب پل پیروزی دادند؟

  answer:
  ایران




In [ ]:
results = {"context":[],"question":[],"correct_ans":[],"model_ans":[]}
for i in range(177,200):
  context = samples[i]['context']
  question = samples[i]['question']
  correct_ans =  samples[i]['answers']
  print(f'question {i+1}')

  prompt = f"""
  In this task, you will be shown a Persian passage and question. You need to write a answer for the question. Try to keep your answers as short as possible.


  samples:
  {sample_string}
  context:
  '''{context}'''

  question:
  '''{question}'''

  answer:
  """

  response = get_completion(prompt)
  print(prompt)
  print("model ans: ",response)
  print("correct ans: ",correct_ans)
  print("-------------------------------------------------------------------------------------")
  results['context'].append(context)
  results['question'].append(question)
  results['correct_ans'].append(correct_ans)
  results['model_ans'].append(response)

  time.sleep(22)

question 178

  In this task, you will be shown a Persian passage and question. You need to write a answer for the question. Try to keep your answers as short as possible.


  samples:
  
  question:
  ویتامین ای را چه موقع استفاده کنیم؟

  answer:
  برای رفع چین و چروک پیشانی و دیگر نشانه های پیری


  question:
  چرا نام برج میلاد میلاد است؟

  answer:
  به مناسبت یکصدمین زادروز روح‌الله خمینی بنیان‌گذار جمهوری اسلامی ایران


  question:
  متفقین به کدام کشور لقب پل پیروزی دادند؟

  answer:
  ایران


  context:
  '''جز هرمزگان و استان خراسان رضوی، استان‌های خوزستان، کردستان، آذربایجان های غربی و شرقی، بوشهر و کرمانشاه هم در وضعیت قرمز و استان‌های اصفهان، همدان، زنجان، کرمان، خراسان شمالی و کهگیلویه و بویراحمد در وضعیت هشدار قرار دارند.'''

  question:
  '''در کدام استان ها وضعیت قرمز است؟'''

  answer:
  
model ans:  هرمزگان و استان خراسان رضوی
correct ans:  ['هرمزگان و استان خراسان رضوی', 'خوزستان، کردستان، آذربایجان های غربی و شرقی، بوشهر و کرمانشاه']
-------------------------------

In [ ]:
files = ['english-prompt-3shot-150-178.csv','english-prompt-3shot-178-200.csv']
df = pd.read_csv('english-prompt-3shot-1-150.csv',index_col=[0])
for file in files:
  df1 = pd.read_csv(file,index_col=[0])
  df = pd.concat([df, df1], axis=0)

In [ ]:
df.to_csv('english-prompt-3shot.csv')

persian prompt

In [ ]:
results = {"context":[],"question":[],"correct_ans":[],"model_ans":[]}
for i in range(0,200):
  context = samples[i]['context']
  question = samples[i]['question']
  correct_ans =  samples[i]['answers']
  print(f'question {i+1}')

  prompt = f"""
  در ادامه به شما یک متن فارسی و یک سوال نشان داده می شود. شما باید برای سوال یک پاسخ بنویسید. سعی کنید پاسخ های خود را تا حد ممکن کوتاه بدهید.

  متن:
  '''{context}'''

  سوال:
  '''{question}'''

  جواب:
  """

  response = get_completion(prompt)
  print(prompt)
  print("model ans: ",response)
  print("correct ans: ",correct_ans)
  print("-------------------------------------------------------------------------------------")
  results['context'].append(context)
  results['question'].append(question)
  results['correct_ans'].append(correct_ans)
  results['model_ans'].append(response)

  time.sleep(22)



question 1

  در ادامه به شما یک متن فارسی و یک سوال نشان داده می شود. شما باید برای سوال یک پاسخ بنویسید. سعی کنید پاسخ های خود را تا حد ممکن کوتاه بدهید.
 
  متن:
  '''هیدروسفالی(به انگلیسی: Hydrocephalus) در پزشکی به وضعیتی گفته می‌شود که مایع مغزی-نخاعی تولید شده در شبکه کوروئید٬ بیش از اندازه در بطن‌های مغزی و دیگر حفره‌های مغز انباشته گردد و منجر به افزایش حجم آن‌ها شود. این افزایش حجم اشغال شده توسط مایع در سیستم عصبی مرکزی و بویژه مغز می‌تواند ناشی از اختلال در توزیع٬ تشکیل٬ جریان یا جذب مایع مغزی-نخاعی باشد و پی‌آمد آن افزایش فشار درون جمجمه و آسیب‌های مغزی و حتی مرگ است. هیدروسفالی در کودکان زیر دو سال به‌خاطر بازبودن سوچورهای کرانیکال(درزهای استخوان جمجمه) منجربه بزرگ شدن غیرطبیعی جمجمه شده ولی در بزرگ‌سالان با اختلال در راه رفتن و حافظه نمود میابد. هیدروسفالی در کودکان بیشتر بعلت ناهنجاری مادرزادی کیاری*٬ تنگی در مجرای مغزی(سیلویوس) مادرزادی یا خونریزی و عفونت نوزادی داخل بطنی پدید می‌آید. بزرگ شدن بالای سر همراه‌با نازک شدن استخوان‌های جمجمه و جدا شدن سوچورها و در موارد پی

In [ ]:
df = pd.DataFrame(results)
df.to_csv('persian-prompt-zero.csv')

persian prompt - 1shot

In [ ]:
template = f"""
  سوال:
  Q

  جواب:
  ANS

"""
sample_string = ''.join(template.replace('Q',s['question']).replace('ANS',s['answers'][0]) for s in three_shot[:1])
print(sample_string)


  سوال:
  ویتامین ای را چه موقع استفاده کنیم؟

  جواب:
  برای رفع چین و چروک پیشانی و دیگر نشانه های پیری




In [ ]:
results = {"context":[],"question":[],"correct_ans":[],"model_ans":[]}
for i in range(0,200):
  context = samples[i]['context']
  question = samples[i]['question']
  correct_ans =  samples[i]['answers']
  print(f'question {i+1}')

  prompt = f"""
  در ادامه به شما یک متن فارسی و یک سوال نشان داده می شود. شما باید برای سوال یک پاسخ بنویسید. سعی کنید پاسخ های خود را تا حد ممکن کوتاه بدهید.

  نمونه:
  {sample_string}
  متن:
  '''{context}'''

  سوال:
  '''{question}'''

  جواب:
  """

  response = get_completion(prompt)
  print(prompt)
  print("model ans: ",response)
  print("correct ans: ",correct_ans)
  print("-------------------------------------------------------------------------------------")
  results['context'].append(context)
  results['question'].append(question)
  results['correct_ans'].append(correct_ans)
  results['model_ans'].append(response)

  time.sleep(22)

question 1

  در ادامه به شما یک متن فارسی و یک سوال نشان داده می شود. شما باید برای سوال یک پاسخ بنویسید. سعی کنید پاسخ های خود را تا حد ممکن کوتاه بدهید.

  نمونه:
  
  سوال:
  ویتامین ای را چه موقع استفاده کنیم؟

  جواب:
  برای رفع چین و چروک پیشانی و دیگر نشانه های پیری


  متن:
  '''هیدروسفالی(به انگلیسی: Hydrocephalus) در پزشکی به وضعیتی گفته می‌شود که مایع مغزی-نخاعی تولید شده در شبکه کوروئید٬ بیش از اندازه در بطن‌های مغزی و دیگر حفره‌های مغز انباشته گردد و منجر به افزایش حجم آن‌ها شود. این افزایش حجم اشغال شده توسط مایع در سیستم عصبی مرکزی و بویژه مغز می‌تواند ناشی از اختلال در توزیع٬ تشکیل٬ جریان یا جذب مایع مغزی-نخاعی باشد و پی‌آمد آن افزایش فشار درون جمجمه و آسیب‌های مغزی و حتی مرگ است. هیدروسفالی در کودکان زیر دو سال به‌خاطر بازبودن سوچورهای کرانیکال(درزهای استخوان جمجمه) منجربه بزرگ شدن غیرطبیعی جمجمه شده ولی در بزرگ‌سالان با اختلال در راه رفتن و حافظه نمود میابد. هیدروسفالی در کودکان بیشتر بعلت ناهنجاری مادرزادی کیاری*٬ تنگی در مجرای مغزی(سیلویوس) مادرزادی یا خونریزی و عف

persian prompt - 3shot

In [ ]:
template = f"""
  سوال:
  Q

  جواب:
  ANS

"""
sample_string = ''.join(template.replace('Q',s['question']).replace('ANS',s['answers'][0]) for s in three_shot)

In [ ]:
print(sample_string)


  سوال:
  ویتامین ای را چه موقع استفاده کنیم؟

  جواب:
  برای رفع چین و چروک پیشانی و دیگر نشانه های پیری


  سوال:
  چرا نام برج میلاد میلاد است؟

  جواب:
  به مناسبت یکصدمین زادروز روح‌الله خمینی بنیان‌گذار جمهوری اسلامی ایران


  سوال:
  متفقین به کدام کشور لقب پل پیروزی دادند؟

  جواب:
  ایران




In [ ]:
results = {"context":[],"question":[],"correct_ans":[],"model_ans":[]}
for i in range(176,200):
  context = samples[i]['context']
  question = samples[i]['question']
  correct_ans =  samples[i]['answers']
  print(f'question {i+1}')

  prompt = f"""
  در ادامه به شما یک متن فارسی و یک سوال نشان داده می شود. شما باید برای سوال یک پاسخ بنویسید. سعی کنید پاسخ های خود را تا حد ممکن کوتاه بدهید.

  نمونه:
  {sample_string}
  متن:
  '''{context}'''

  سوال:
  '''{question}'''

  جواب:
  """

  response = get_completion(prompt)
  print(prompt)
  print("model ans: ",response)
  print("correct ans: ",correct_ans)
  print("-------------------------------------------------------------------------------------")
  results['context'].append(context)
  results['question'].append(question)
  results['correct_ans'].append(correct_ans)
  results['model_ans'].append(response)

  time.sleep(22)

question 177

  در ادامه به شما یک متن فارسی و یک سوال نشان داده می شود. شما باید برای سوال یک پاسخ بنویسید. سعی کنید پاسخ های خود را تا حد ممکن کوتاه بدهید.
 
  نمونه:
  
  سوال:
  ویتامین ای را چه موقع استفاده کنیم؟

  جواب:
  برای رفع چین و چروک پیشانی و دیگر نشانه های پیری


  سوال:
  چرا نام برج میلاد میلاد است؟

  جواب:
  به مناسبت یکصدمین زادروز روح‌الله خمینی بنیان‌گذار جمهوری اسلامی ایران


  سوال:
  متفقین به کدام کشور لقب پل پیروزی دادند؟

  جواب:
  ایران


  متن:
  '''مساحت خلیج فارس ۲۳۷٬۴۷۳ کیلومتر مربع است، طول آن حدود ۹۰۰ کیلومتر و عرض متوسط آن ۲۵۰ کیلومتر است و پس از خلیج مکزیک و خلیج هادسون سومین خلیج بزرگ جهان بشمار می‌آید. خلیج فارس از شرق از طریق تنگه هرمز و دریای عمان به اقیانوس هند و دریای عرب راه دارد، و از غرب به دلتای رودخانه اروندرود، که حاصل پیوند دو رودخانهٔ دجله و فرات و پیوستن رود کارون به آن است، ختم می‌شود. هشت کشور در اطراف خلیج فارس قرار دارند. کشورهای ایران، عمان، عراق، عربستان سعودی، کویت، امارات متحده عربی، قطر و بحرین در کناره خلیج فارس هستند. در ا

In [ ]:
df = pd.DataFrame(results)
df.to_csv('persian-prompt-3shot-177-200')

In [ ]:
files = ['persian-prompt-3shot-36-46','persian-prompt-3shot-46-177','persian-prompt-3shot-177-200']
df = pd.read_csv('persian-prompt-3shot-1-35',index_col=[0])
for file in files:
  df1 = pd.read_csv(file,index_col=[0])
  df = pd.concat([df, df1], axis=0)

In [ ]:
df.to_csv('persian-prompt-3shot.csv')

In [3]:
# these functions are heavily influenced by the HF squad_metrics.py script
def preprocess_text(s,normalizer):

    def normalize(text):
        text.replace("پاسخ:","")
        return normalizer.normalize(text)

    def remove_punc_stopword(text):
        exclude = ["?",".","!","؟",":","،",")","(","..."]
        return "".join(ch for ch in text if ch not in exclude)

    return normalize(remove_punc_stopword(s))

def compute_exact_match(prediction, truth):
    return int(preprocess_text(prediction) == preprocess_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = prediction.split()
    truth_tokens = truth.split()

    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)

    common_tokens = set(pred_tokens) & set(truth_tokens)

    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0

    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)

    return 2 * (prec * rec) / (prec + rec)

def get_gold_answers(i):
    """helper function that retrieves all possible true answers from a squad2.0 example"""

    gold_answers = [answer["text"] for answer in example.answers if answer["text"]]

    # if gold_answers doesn't exist it's because this is a negative example -
    # the only correct answer is an empty string
    if not gold_answers:
        gold_answers = [""]

    return gold_answers

In [4]:
def calculate_f1(model_ans_list,correct_ans_list):
  normalizer = Normalizer()
  f1 = []
  for i in range(0,len(model_ans_list)):
    model_ans = preprocess_text(model_ans_list[i],normalizer)
    correct_ans =  [preprocess_text(text,normalizer) for text in correct_ans_list[i]]
    f1.append(max((compute_f1(model_ans, answer)) for answer in correct_ans))
    #print(f'model: \n {model_ans}')
    #print(f'truth: \n {correct_ans}')
    #print(f'F1: {f1[i]}')
  return np.array(f1).mean()


## GPT

### english prompt

zero

In [ ]:
df = pd.read_csv('english-prompt-zero.csv',index_col=[0])
gpt_results = df['model_ans'].values
true_ans = [sample['answers'] for sample in samples]
f1 = calculate_f1(gpt_results,true_ans)
print(f'average f1: {f1}')

average f1: 0.5887405969326296


1- shot

In [19]:
df = pd.read_csv('rc-english-prompt-1shot.csv',index_col=[0])
gpt_results = df['model_ans'].values
true_ans = [sample['answers'] for sample in samples]
f1 = calculate_f1(gpt_results,true_ans)
print(f'average f1: {f1}')

average f1: 0.6440975155186797


3-shot

In [ ]:
df = pd.read_csv('english-prompt-3shot.csv',index_col=[0])
gpt_results = df['model_ans'].values
true_ans = [sample['answers'] for sample in samples]
f1 = calculate_f1(gpt_results,true_ans)
print(f'average f1: {f1}')

average f1: 0.6442939177087413


### persian prompt

zero

In [ ]:
df = pd.read_csv('persian-prompt-zero.csv',index_col=[0])
gpt_results = df['model_ans'].values
true_ans = [sample['answers'] for sample in samples]
f1 = calculate_f1(gpt_results,true_ans)
print(f'average f1: {f1}')

average f1: 0.5359483253416455


1-shot

In [20]:
df = pd.read_csv('rc-persian-prompt-1shot.csv',index_col=[0])
gpt_results = df['model_ans'].values
true_ans = [sample['answers'] for sample in samples]
f1 = calculate_f1(gpt_results,true_ans)
print(f'average f1: {f1}')

average f1: 0.64327839330186


3-shot

In [22]:
df = pd.read_csv('persian-prompt-3shot.csv',index_col=[0])
gpt_results = df['model_ans'].values
true_ans = [sample['answers'] for sample in samples]
f1 = calculate_f1(gpt_results,true_ans)
print(f'average f1: {f1}')

average f1: 0.6425398281311803


# MT5

In [ ]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer

def run_model(paragraph, question, **generator_args):
    input_ids = tokenizer.encode(question + "\n" + paragraph, return_tensors="pt")
    res = model.generate(input_ids, **generator_args)
    output = tokenizer.batch_decode(res, skip_special_tokens=True)
    return output

## mt5 small

In [ ]:
model_size = "small"
model_name = f"persiannlp/mt5-{model_size}-parsinlu-squad-reading-comprehension"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
small_ans_list = [run_model(sample['context'], sample['question']) for sample in samples]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
small_results = np.array(small_ans_list).flatten()
true_ans = [sample['answers'] for sample in samples]
f1 = calculate_f1(small_results,true_ans)
print(f'average f1: {f1}')

model: 
 پزشکی به وضعیتی گفته می‌شود که مایع مغزی-نخ
truth: 
 ['به وضعیتی گفته می\u200cشود که مایع مغزی-نخاعی تولید شده در شبکه کوروئید٬ بیش از اندازه در بطن\u200cهای مغزی و دیگر حفره\u200cهای مغز انباشته گردد و منجر به افزایش حجم آن\u200cها شود', 'مایع مغزی-نخاعی تولید شده در شبکه کوروئید٬ بیش از اندازه در بطن\u200cهای مغزی و دیگر حفره\u200cهای مغز انباشته گردد و منجر به افزایش حجم آن\u200cها شود']
F1: 0.30769230769230765
model: 
 مجارستان
truth: 
 ['مجارستان']
F1: 1.0
model: 
 فرانسه آلمان و بریتانیا
truth: 
 ['فرانسه آلمان و بریتانیا']
F1: 1.0
model: 
 هفتاد و پنج روز بعد از درگذشت محمد و دیگری نود و پنج روز
truth: 
 ['شیعیان پیرامون تاریخ درگذشت فاطمه دو روایت متفاوت شامل هفتاد و پنج روز بعد از درگذشت محمد و دیگری نود و پنج روز را مورد وثوق می\u200cدانند']
F1: 0.4878048780487805
model: 
 نیرویی که امتداد آنها از نقطه عبور می‌کند گشتاور
truth: 
 ['نیروهایی که امتداد آنها از نقطه عبور می\u200cکند گشتاور نیرویی نسبت به این نقطه ندارند بنابراین نیرویی که تکیه\u200cگاه بر میله وارد می\u

## mt5 base

In [ ]:
model_size = "base"
model_name = f"persiannlp/mt5-{model_size}-parsinlu-squad-reading-comprehension"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
base_ans_list = [run_model(sample['context'], sample['question']) for sample in samples]

In [ ]:
base_results = np.array(base_ans_list).flatten()
true_ans = [sample['answers'] for sample in samples]
f1 = calculate_f1(base_results,true_ans)
print(f'average f1: {f1}')

model: 
 هیدروسفالی در کودکان زیر دو سال به خاطر بازبودن سوچورهای کر
truth: 
 ['به وضعیتی گفته می\u200cشود که مایع مغزی-نخاعی تولید شده در شبکه کوروئید٬ بیش از اندازه در بطن\u200cهای مغزی و دیگر حفره\u200cهای مغز انباشته گردد و منجر به افزایش حجم آن\u200cها شود', 'مایع مغزی-نخاعی تولید شده در شبکه کوروئید٬ بیش از اندازه در بطن\u200cهای مغزی و دیگر حفره\u200cهای مغز انباشته گردد و منجر به افزایش حجم آن\u200cها شود']
F1: 0.10810810810810811
model: 
 مجارستان
truth: 
 ['مجارستان']
F1: 1.0
model: 
 فرانسه آلمان و بریتانیا
truth: 
 ['فرانسه آلمان و بریتانیا']
F1: 1.0
model: 
 هفتاد و پنج روز بعد از درگذشت محمد و دیگری نود و پنج روز
truth: 
 ['شیعیان پیرامون تاریخ درگذشت فاطمه دو روایت متفاوت شامل هفتاد و پنج روز بعد از درگذشت محمد و دیگری نود و پنج روز را مورد وثوق می\u200cدانند']
F1: 0.4878048780487805
model: 
 نیرویی که تکیه‌گاه بر میله وارد می‌کند دارای گشت
truth: 
 ['نیروهایی که امتداد آنها از نقطه عبور می\u200cکند گشتاور نیرویی نسبت به این نقطه ندارند بنابراین نیرویی که تکیه\u200cگاه ب

## mt5 large

In [ ]:
model_size = "large"
model_name = f"persiannlp/mt5-{model_size}-parsinlu-squad-reading-comprehension"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
large_ans_list = [run_model(sample['context'], sample['question']) for sample in tqdm(samples)]

  0%|          | 0/200 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 200/200 [12:45<00:00,  3.83s/it]


In [ ]:
large_results = np.array(large_ans_list).flatten()
true_ans = [sample['answers'] for sample in samples]
f1 = calculate_f1(large_results,true_ans)
print(f'average f1: {f1}')

model: 
 هیدروسفالیبه انگلیسی Hydrocephalus در پزشکی به
truth: 
 ['به وضعیتی گفته می\u200cشود که مایع مغزی-نخاعی تولید شده در شبکه کوروئید٬ بیش از اندازه در بطن\u200cهای مغزی و دیگر حفره\u200cهای مغز انباشته گردد و منجر به افزایش حجم آن\u200cها شود', 'مایع مغزی-نخاعی تولید شده در شبکه کوروئید٬ بیش از اندازه در بطن\u200cهای مغزی و دیگر حفره\u200cهای مغز انباشته گردد و منجر به افزایش حجم آن\u200cها شود']
F1: 0.125
model: 
 مجارستان
truth: 
 ['مجارستان']
F1: 1.0
model: 
 فرانسه آلمان و بریتانیا
truth: 
 ['فرانسه آلمان و بریتانیا']
F1: 1.0
model: 
 هفتاد و پنج روز بعد از درگذشت محمد و دیگری نود و پنج روز
truth: 
 ['شیعیان پیرامون تاریخ درگذشت فاطمه دو روایت متفاوت شامل هفتاد و پنج روز بعد از درگذشت محمد و دیگری نود و پنج روز را مورد وثوق می\u200cدانند']
F1: 0.4878048780487805
model: 
 نیرویی که تکیه‌گاه بر میله وارد می‌کند
truth: 
 ['نیروهایی که امتداد آنها از نقطه عبور می\u200cکند گشتاور نیرویی نسبت به این نقطه ندارند بنابراین نیرویی که تکیه\u200cگاه بر میله وارد می\u200cکند دارای گشتاور 

In [ ]:
df = pd.read_csv('result.csv').drop('Unnamed: 0',axis = 1)
df

,context,question,correct_ans,gpt,mt5-small,mt5-base,mt5-large
0,هیدروسفالی(به انگلیسی: Hydrocephalus) در پزشکی...,هیدروسفالی چه نوع بیماری است؟,['به وضعیتی گفته می\u200cشود که مایع مغزی-نخاع...,هیدروسفالی یک بیماری در سیستم عصبی مرکزی است ک...,پزشکی به وضعیتی گفته می شود که مایع مغزی-نخ,هیدروسفالی در کودکان زیر دو سال به خاطر بازبود...,هیدروسفالی(به انگلیسی: Hydrocephalus) در پزشکی به
1,بوداپـِست پایتخت، بزرگترین و پرجمعیت‌ترین شهر ...,بوداپست در کدام کشور است؟,['مجارستان'],بوداپست در کشور مجارستان است.,مجارستان,مجارستان,مجارستان
2,سه کشور اروپایی (EU 3) یا سه کشور بزرگ اروپا (...,3eu کدام کشور است؟,['فرانسه، آلمان و بریتانیا'],این سوال قابل پاسخ نیست، چرا که متن اشاره به ک...,فرانسه، آلمان و بریتانیا,فرانسه، آلمان و بریتانیا,فرانسه، آلمان و بریتانیا
3,شیعیان پیرامون تاریخ درگذشت فاطمه دو روایت متف...,چرا دو ایام فاطمیه داریم؟,['شیعیان پیرامون تاریخ درگذشت فاطمه دو روایت م...,دو ایام فاطمیه به دلیل دو روایت متفاوت درباره ...,هفتاد و پنج روز بعد از درگذشت محمد و دیگری نود...,هفتاد و پنج روز بعد از درگذشت محمد و دیگری نود...,هفتاد و پنج روز بعد از درگذشت محمد و دیگری نود...
4,نیروهایی که امتداد آنها از نقطه عبور می‌کند گش...,چه زمانی گشتاور صفر است؟,['نیروهایی که امتداد آنها از نقطه عبور می\u200...,گشتاور صفر زمانی است که نیرویی که تکیه گاه بر ...,نیرویی که امتداد آنها از نقطه عبور می کند گشتاور,نیرویی که تکیه گاه بر میله وارد می کند دارای گشت,نیرویی که تکیه گاه بر میله وارد می کند
...,...,...,...,...,...,...,...
195,حَجّ (ملاقات خانه خدا) سنت ابراهیم است و یکی ا...,انواع حج چیست؟,"['سه نوع تمتع، قران و افراد', 'تمتع، قران و اف...",انواع حج شامل حج تمتع، حج قران و حج افراد است.,ابراهیم,سه نوع تمتع، قران و افراد است,سه نوع تمتع، قران و افراد
196,ختران تازه متولد شده حدود ۱۰۰۰٬۰۰۰ (یک میلیون)...,فولیکول چه زمانی ازاد می شود؟,['هر ماه یک تخمک به مدت ۱۴ روز در فولیکول خودش...,فولیکول پاره شده و تخمک آزاد شده از تخمدان به ...,هر ماه یک تخمک به مدت ۱۴ روز در فولیکول خودش ر...,۱۴ روز,هر ماه یک تخمک به مدت ۱۴ روز در فولیکول خودش ر...
197,طبق تحقیقاتی که در انجمن پزشکی آمریکا صورت گرف...,در داخل لانه خفاش چه چیزی وجود دارد؟,"['بزاق دهان خفاش', 'آنزیمی به نام ""دسموتاپلاز ...",در داخل لانه خفاش، بزاق خفاش وجود دارد که حاوی...,دسموتاپلاز یا DSPA,بزاق استفاده شده در لانه گونه خاصی از خفاش,"بزاق دهان خفاش ها آنزیمی به نام ""دسموتاپلاز"
198,جمعیت یهودیان در جهان، تا ابتدای سال ۲۰۱۳ حدود...,کدام کشور ها یهودی هستند؟,['اسرائیل'],متن این اطلاعات را نمی‌دهد. اما در متن ذکر شده...,اسرائیل و ایالات متحده آمریکا,اسرائیل و ایالات متحده آمریکا,اسرائیل و ایالات متحده آمریکا
